In [1]:
import pandas as pd
import json
import os
import random
from tqdm import tqdm


In [2]:
#function upper string
lambda_upper = lambda x: x.upper()
def mutate_dna_sequence(sequence, mutation_probability):
    mutated_sequence = ""
    
    for base in sequence:
        if random.random() < mutation_probability:
            mutated_base = random.choice(['A', 'T', 'C', 'G'])
            while mutated_base == base:
                mutated_base = random.choice(['A', 'T', 'C', 'G'])
            mutated_sequence += mutated_base
        else:
            mutated_sequence += base
    
    return mutated_sequence

def split_and_mutate_sequence(sequence):
    # Calculate the lengths of each part
    length_1 = int(len(sequence) * 0.06)
    length_2 = int(len(sequence) * 0.62)
    length_3 = int(len(sequence) * 0.29)
    
    # Split the sequence into three parts
    part_1 = sequence[:length_1]
    part_2 = sequence[length_1:length_1+length_2]
    part_3 = sequence[length_1+length_2:]
    
    # Apply mutate_dna_sequence to each part with respective probabilities
    mutated_part_1 = mutate_dna_sequence(part_1, 0.024)
    mutated_part_2 = mutate_dna_sequence(part_2, 0.24)
    mutated_part_3 = mutate_dna_sequence(part_3, 0.008)
    
    # Concatenate the mutated parts into a new sequence
    mutated_sequence = mutated_part_1 + mutated_part_2 + mutated_part_3
    
    # Return the mutated sequence
    return mutated_sequence


In [3]:
# data no need exclude genus 

data_curaca = pd.read_csv(r"C:\Users\Auguste Verdier\Desktop\TeleoClassification\scripts\BERTeleo\experiments\masking_training\data\teleo\all_source\gbif_ma_cu_curaca_2020_teleo.csv")[['DNA_sequence']].rename(columns={'DNA_sequence':'sequence'})
data_evhoe = pd.read_csv(r"C:\Users\Auguste Verdier\Desktop\TeleoClassification\scripts\BERTeleo\experiments\masking_training\data\teleo\all_source\gbif_ma_fr_evhoe_2020_teleo.csv")[['DNA_sequence']].rename(columns={'DNA_sequence':'sequence'})
data_fjord = pd.read_csv(r"C:\Users\Auguste Verdier\Desktop\TeleoClassification\scripts\BERTeleo\experiments\masking_training\data\teleo\all_source\obitools_teleo_arctic_clean_fjord.csv")[['sequence']]
data_fjord['sequence'] = data_fjord['sequence'].apply(lambda_upper)
data_fb_fail = pd.read_csv(r"C:\Users\Auguste Verdier\Desktop\TeleoClassification\scripts\BERTeleo\experiments\masking_training\data\teleo\all_source\teleo_missing_taxa_pga_3_fb.tsv", sep="\t",header = None)
data_gb_fail = pd.read_csv(r"C:\Users\Auguste Verdier\Desktop\TeleoClassification\scripts\BERTeleo\experiments\masking_training\data\teleo\all_source\teleo_missing_taxa_pga_3_gb.tsv", sep="\t",header = None)
data_fb_fail.rename(columns={0:'ID',1:'sequence'}, inplace=True)
data_gb_fail.rename(columns={0:'ID',1:'sequence'}, inplace=True)

# exclude genus from eache  fold

data_fb = pd.read_csv(r"C:\Users\Auguste Verdier\Desktop\TeleoClassification\scripts\BERTeleo\experiments\masking_training\data\teleo\all_source\output_pga_3_fb.tsv", sep="\t",header = None)
data_gb = pd.read_csv(r"C:\Users\Auguste Verdier\Desktop\TeleoClassification\scripts\BERTeleo\experiments\masking_training\data\teleo\all_source\output_pga_3_gb.tsv", sep="\t",header = None)

data_fb.rename(columns={0:'ID_ncbi', 1:'ID', 2:'kingdom', 3:'phylum', 4:'class', 5:'order', 6:'family', 7:'genus', 8:'species', 9:'sequence'}, inplace=True)
data_gb.rename(columns={0:'ID_ncbi', 1:'ID', 2:'kingdom', 3:'phylum', 4:'class', 5:'order', 6:'family', 7:'genus', 8:'species', 9:'sequence'}, inplace=True)



for k in range(1,7):
    fold_genus = json.load(open(fr"C:\Users\Auguste Verdier\Desktop\TeleoClassification\data\6_fold_teleo_aug\fold_{k}\test_genus.json"))
    data_fb['sequence'] = data_fb['sequence'].apply(lambda_upper)
    data_gb['sequence'] = data_gb['sequence'].apply(lambda_upper)
    
    no_genus_fb = data_fb[~data_fb['genus'].isin(fold_genus)]
    no_genus_gb = data_gb[~data_gb['genus'].isin(fold_genus)]

    val_fb = data_fb[data_fb['genus'].isin(fold_genus)]
    val_gb = data_gb[data_gb['genus'].isin(fold_genus)]
    val_set = pd.concat([val_fb[['sequence']], val_gb[['sequence']]], ignore_index=True)

    val_set.drop_duplicates(subset=['sequence'], inplace=True)
    val_set.reset_index(drop=True, inplace=True)

    concat_all = pd.concat([no_genus_fb[['sequence']], no_genus_gb[[ 'sequence']], data_fb_fail[[ 'sequence']], data_curaca, data_evhoe, data_fjord], ignore_index=True)
    concat_all.drop_duplicates(subset=['sequence'], inplace=True)
    concat_all.reset_index(drop=True, inplace=True)
    os.makedirs(r"C:\Users\Auguste Verdier\Desktop\TeleoClassification\scripts\BERTeleo\experiments\masking_training\data\teleo\fold_"+str(k), exist_ok=True)
    concat_all.to_csv(r"C:\Users\Auguste Verdier\Desktop\TeleoClassification\scripts\BERTeleo\experiments\masking_training\data\teleo\fold_"+str(k)+r"\all_sequence.csv", index=False)
    df = concat_all.copy()
    for i in tqdm(range(20)):

        temp_df = df.copy()
        temp_df['sequence'] = df['sequence'].apply(split_and_mutate_sequence)
        concat_all = pd.concat([concat_all, temp_df], ignore_index=True)
        
    

    
    concat_all.to_csv(r"C:\Users\Auguste Verdier\Desktop\TeleoClassification\scripts\BERTeleo\experiments\masking_training\data\teleo\fold_"+str(k)+r"\train.csv", index=False)
    val_set.to_csv(r"C:\Users\Auguste Verdier\Desktop\TeleoClassification\scripts\BERTeleo\experiments\masking_training\data\teleo\fold_"+str(k)+r"\val.csv", index=False)

    print("fold "+str(k)+" done")







100%|██████████| 20/20 [00:03<00:00,  5.37it/s]


fold 1 done


100%|██████████| 20/20 [00:03<00:00,  5.36it/s]


fold 2 done


100%|██████████| 20/20 [00:03<00:00,  5.36it/s]


...

fold 4 done


100%|██████████| 20/20 [00:03<00:00,  5.37it/s]


fold 5 done


100%|██████████| 20/20 [00:03<00:00,  5.36it/s]


fold 6 done
